In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
from package.base import DSPyChoice, DSPyResult, DriverLM

In [60]:
import requests

def llama_prompt_fn(messages: list[dict]) -> str:
    # DSPy puts the full formatted prompt in the last message
    return messages[-1]['content'] if messages else ""


def llama_request_fn(prompt: str, temperature: float = 0.0, max_tokens: int | None = None) -> dict:
    # model = "llama3.2:1b"
    model = "llama3.2-vision:11b"
    response = requests.post(
        'http://localhost:11434/api/chat',  # Changed to /chat
        json={
            "model": model,
            "messages": [{"role": "user", "content": prompt}],  # Changed format
            "stream": False,
            "options": {"temperature": temperature}
        }
    )
    response.raise_for_status()
    data = response.json()
    # Extract text from chat response
    return {"response": data["message"]["content"]}

def llama_output_fn(response:dict)->DSPyResult:
    text = response.get("response", "").strip()
    return DSPyResult(text)

In [38]:
prompt = llama_prompt_fn([{"role":"user","content":"hello"}])
response = llama_request_fn(prompt)
response = llama_output_fn(response)
response

{'choices': [<package.base.DSPyChoice at 0x209f9c1a090>]}

In [39]:
# response['choices'][0].message['content']

In [40]:
lm = DriverLM(
    request_fn=llama_request_fn,
    prompt_fn=llama_prompt_fn,
    output_fn=llama_output_fn,
    temperature=0,
    max_tokens=500
)

In [41]:
lm.forward(messages=[{"role":"user", "content":"hello"}])

{'choices': [<package.base.DSPyChoice at 0x209f9e31790>]}

In [42]:
lm(messages=[{"role":"user", "content":"hello"}])

['Hello! How can I assist you today?']

In [43]:
import dspy

# Configure DSPy with Ollama
lm = dspy.LM(
    model="ollama/llama3.2:1b",
    api_base="http://localhost:11434",
    temperature=0.0,
    max_tokens=500
)
dspy.configure(lm=lm)

# Use it
response = lm(messages=[{"role": "user", "content": "hello"}])
print(response)

# Or with DSPy modules
predict = dspy.Predict("question -> answer")
result = predict(question="What is 2+2?")
print(result.answer)

['Hello. How can I assist you today?']
2


In [44]:
result

Prediction(
    answer='2'
)

In [45]:
lm = DriverLM(
    request_fn=llama_request_fn,
    prompt_fn=llama_prompt_fn,
    output_fn=llama_output_fn,
    temperature=0,
    max_tokens=500
)
dspy.configure(lm=lm)

# Use it
response = lm(messages=[{"role": "user", "content": "hello"}])
print(response)

# Or with DSPy modules
predict = dspy.Predict("question -> answer")
result = predict(question="What is 2+2?")
print(result.answer)


['Hello! How can I assist you today?']
4


In [46]:
result

Prediction(
    answer='4'
)

In [47]:
import dspy

# Configure with your DriverLM
lm = DriverLM(
    request_fn=llama_request_fn,
    prompt_fn=llama_prompt_fn,
    output_fn=llama_output_fn,
    temperature=0,
    max_tokens=500
)
dspy.configure(lm=lm)

# Define a signature
class QA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Use it
qa = dspy.Predict(QA)
result = qa(question="What is the capital of France?")
print(f"Answer: {result.answer}")
result

Answer: Paris


Prediction(
    answer='Paris'
)

In [48]:
# Try with ChainOfThought
cot = dspy.ChainOfThought(QA)
result2 = cot(question="What is 15 * 23?")
# print(f"Reasoning: {result2.rationale}")
# print(f"Answer: {result2.answer}")
result2

Prediction(
    reasoning='To find the product of 15 and 23, we can simply multiply these two numbers together.\n\n15 * 23 = 345',
    answer='345'
)

In [49]:
# Test with native dspy.LM
lm = dspy.LM(
    model="ollama/llama3.2:1b",
    api_base="http://localhost:11434",
    temperature=0.0,
    max_tokens=500
)
dspy.configure(lm=lm)

cot = dspy.ChainOfThought(QA)
result = cot(question="What is 15 * 23?")

# Check if reasoning exists
print("Keys:", result.keys())
result

Keys: ['reasoning', 'answer']


Prediction(
    reasoning='This multiplication problem can be solved by simply multiplying the numbers together.',
    answer='The result of this calculation is 345.'
)

In [67]:
import dspy
from dspy.teleprompt import BootstrapFewShot

# Native LM from dspy
lm = dspy.LM(
    model="ollama/llama3.2-vision:11b",
    api_base="http://localhost:11434",
    temperature=0.0,
    max_tokens=500
)
dspy.configure(lm=lm)

# Define signature
class Emotion(dspy.Signature):
    """Classify the emotion of a sentence."""
    sentence = dspy.InputField()
    emotion = dspy.OutputField(desc="happy, sad, or neutral")

# Create training data
trainset = [
    dspy.Example(sentence="I love this!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="This is terrible.", emotion="sad").with_inputs("sentence"),
    dspy.Example(sentence="It's okay.", emotion="neutral").with_inputs("sentence"),
]

# Create module
class EmotionClassifier(dspy.Module):
    def __init__(self):
        self.prog = dspy.ChainOfThought(Emotion)
    
    def forward(self, sentence):
        return self.prog(sentence=sentence)

# Optimize
optimizer = BootstrapFewShot(max_bootstrapped_demos=2, max_labeled_demos=2)
optimized = optimizer.compile(EmotionClassifier(), trainset=trainset)

# Test
result = optimized(sentence="I'm so excited!")
print(f"Emotion: {result.emotion}")
print(f"Reasoning: {result.reasoning}")
optimized(sentence="I love pizza")

 67%|██████▋   | 2/3 [00:00<00:00, 49.17it/s]

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Emotion: happy
Reasoning: The sentence contains the word "excited", which is typically associated with positive emotions.


Prediction(
    reasoning='The sentence contains the word "love", which is typically associated with positive emotions, and the word "pizza" is a common topic of enjoyment.',
    emotion='happy'
)

In [66]:
import dspy
from dspy.teleprompt import BootstrapFewShot

# Configure your DriverLM
lm = DriverLM(
    request_fn=llama_request_fn,
    prompt_fn=llama_prompt_fn,
    output_fn=llama_output_fn,
    temperature=0,
    max_tokens=500
)
dspy.configure(lm=lm)

# Define signature
class Emotion(dspy.Signature):
    """Classify the emotion of a sentence."""
    sentence = dspy.InputField()
    emotion = dspy.OutputField(desc="happy, sad, or neutral")

# Create training data
trainset = [
    dspy.Example(sentence="I love this!", emotion="happy").with_inputs("sentence"),
    dspy.Example(sentence="This is terrible.", emotion="sad").with_inputs("sentence"),
    dspy.Example(sentence="It's okay.", emotion="neutral").with_inputs("sentence"),
]

# Create module
class EmotionClassifier(dspy.Module):
    def __init__(self):
        self.prog = dspy.ChainOfThought(Emotion)
    
    def forward(self, sentence):
        return self.prog(sentence=sentence)

# Optimize
optimizer = BootstrapFewShot(max_bootstrapped_demos=2, max_labeled_demos=2)
optimized = optimizer.compile(EmotionClassifier(), trainset=trainset)

# Test
result = optimized(sentence="I'm so excited!")
print(f"Emotion: {result.emotion}")
print(f"Reasoning: {result.reasoning}")
optimized(sentence="I love pizza")

 67%|██████▋   | 2/3 [00:01<00:00,  1.06it/s]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Emotion: The primary emotion expressed in this sentence is excitement, which is a strong and positive feeling of eagerness or anticipation.
Reasoning: The sentence "I'm so excited!" is a common expression used to convey enthusiasm or eagerness about something, often in a lighthearted or playful manner.


Prediction(
    reasoning='The sentence "I love pizza" is a statement expressing a positive sentiment towards pizza, indicating a strong affection or enjoyment for it.',
    emotion='The emotion expressed in the sentence is love, which is a strong and positive feeling.'
)

In [64]:
optimized(sentence="I love pizza")

Prediction(
    reasoning='The sentence "I love pizza" is a statement expressing a positive sentiment towards pizza, indicating a strong affection or enjoyment for it.',
    emotion='The emotion expressed in the sentence is love, which is a strong and positive feeling.'
)

In [68]:
# After optimization
# optimized = optimizer.compile(EmotionClassifier(), trainset=trainset)

# 1. Inspect the optimized prompts
print("=== Optimized Module ===")
for name, predictor in optimized.named_predictors():
    print(f"\n{name}:")
    print(f"Signature: {predictor.signature}")
    if hasattr(predictor, 'demos'):
        print(f"Demos: {len(predictor.demos)} examples")
        for i, demo in enumerate(predictor.demos):
            print(f"  Demo {i+1}: {demo}")

# 2. Save the optimized module
optimized.save("emotion_classifier.json")
print("\nSaved to emotion_classifier.json")

# 3. Load it later
loaded = EmotionClassifier()
loaded.load("emotion_classifier.json")

# 4. Simple inspection
print("\n=== Quick Check ===")
print(f"Module type: {type(optimized)}")
print(f"Has prog: {hasattr(optimized, 'prog')}")
if hasattr(optimized, 'prog'):
    print(f"Prog demos: {len(optimized.prog.demos) if hasattr(optimized.prog, 'demos') else 0}")


=== Optimized Module ===

prog.predict:
Signature: StringSignature(sentence -> reasoning, emotion
    instructions='Classify the emotion of a sentence.'
    sentence = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Sentence:', 'desc': '${sentence}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    emotion = Field(annotation=str required=True json_schema_extra={'desc': 'happy, sad, or neutral', '__dspy_field_type': 'output', 'prefix': 'Emotion:'})
)
Demos: 2 examples
  Demo 1: Example({'augmented': True, 'sentence': 'I love this!', 'reasoning': 'The sentence contains the word "love", which is typically associated with positive emotions.', 'emotion': 'happy'}) (input_keys=None)
  Demo 2: Example({'augmented': True, 'sentence': 'This is terrible.', 'reasoning': 'The sentence contains the word "terribl